In [7]:
import pandas as pd
import spacy
import numpy as np
from tqdm import tqdm

In [8]:
nlp = spacy.load("en_core_web_lg")

In [16]:
df = pd.read_excel('./sst_test_map_syn_deberta.xlsx')
# 复制数据框并清空数据
df_copy = df.copy()
df_copy = df_copy.iloc[0:0]
entity_list = ['CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']
for entity_kind in entity_list:
    df[entity_kind] = 0
    df[entity_kind + '_text'] = 'blank'
    df_copy[entity_kind] = 0
    df_copy[entity_kind + '_text'] = 'blank'

In [17]:
for index, row in tqdm(df.iterrows()):
    judge_text = row['insert_text']
    doc = nlp(judge_text)
    if len(doc.ents) > 0:
        insert_sentiment = int(row['insert_sentiment'])
        if insert_sentiment == 1:
            new_negative_score = float(row['res_neg_score'])
            original_negative_score = float(row['original_neg_score'])
            if new_negative_score - original_negative_score >= 0.1:
                for ent in doc.ents:
                    row[ent.label_] += 1
                if row[ent.label_] == 1:
                    row[ent.label_ + '_text'] = ent.text
                else:
                    row[ent.label_ + '_text'] += ',' + ent.text
                df_copy = pd.concat([df_copy, row.to_frame().T], axis=0)
        else:
            new_positive_score = float(row['res_pos_score'])
            original_positive_score = float(row['original_pos_score'])
            if new_positive_score - original_positive_score >= 0.1:
                for ent in doc.ents:
                    row[ent.label_] += 1
                if row[ent.label_] == 1:
                    row[ent.label_ + '_text'] = ent.text
                else:
                    row[ent.label_ + '_text'] += ',' + ent.text
                df_copy = pd.concat([df_copy, row.to_frame().T], axis=0)
df_copy.to_csv('result_deberta_entity_sentiment.tsv', sep='\t', index=False)

999it [00:07, 139.99it/s]
